In [ ]:
pip install selenium==4.17.2

In [4]:
from selenium import webdriver

driver = webdriver.Chrome()

driver.get('https://www.lawtalk.co.kr/posts/case')

In [5]:
from selenium.webdriver.common.by import By
import time
import random

random_time = random.uniform(1, 1.5)

## 팝업창 종료 '오늘 그만보기' 클릭
driver.find_element(By.CSS_SELECTOR, 'body > div.hackle-iam > div > div > div > button.hackle-iam__button.css-16ljet3').click()
time.sleep(random_time)

## 분야 선택
driver.find_element(By.CSS_SELECTOR, '#root-view > div > div.search-content > div > section > div.posts-section-wrap > div > div > div.posts-section__search-menu > div > category-select-box > div > div.select-item-wrap > div.select-item').click()
time.sleep(random_time)

## 성범죄
# driver.find_element(By.CSS_SELECTOR, '.select-item-option__radio-items:nth-of-type(2)').click()
# time.sleep(random_time)

## 음주운전
driver.find_element(By.CSS_SELECTOR, '.select-item-option__radio-items:nth-of-type(4)').click()
time.sleep(random_time)

## 최신순 클릭
driver.find_element(By.CSS_SELECTOR, '#root-view > div > div.search-content > div > section > div.posts-section-wrap > div > div.posts-section__body > div.posts-section__search-menu > ul > li:nth-child(2)').click()

무한스크롤 처리

In [6]:
import time
import random

random_time = random.uniform(1, 1.5) 

before_height = driver.execute_script("return document.body.scrollHeight")

while True :
    # 스크롤 끝까지 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(random_time)
    
    # 스크롤 후 높이 확인
    after_height = driver.execute_script("return document.body.scrollHeight")

    if before_height == after_height:
        break
    
    before_height = after_height

나무태그 추출

In [7]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

articles = soup.select('.card-list-section__card-wrap')

result = []
for article in articles[:3]:
    title = article.select_one('.card-list-section__card-wrap .solution-card__title').text.strip()
    law_result = article.select_one('.card-list-section__card-wrap .solution-card__sentence-result-text').text
    link = 'https://www.lawtalk.co.kr/' + article.select_one('.card-list-section__card-wrap .solution-card__link').attrs['href']
    result.append([title, law_result, link])
result

[['[교통사고변호사] 위험운전치상 음주운전 벌금형 성공사례 ✔',
  '벌금형',
  'https://www.lawtalk.co.kr//posts/87800'],
 ['✅음주 재범(수치 0.299) 벌금형 전문직 자격유지 #음주운전',
  '벌금 1000만원',
  'https://www.lawtalk.co.kr//posts/87777'],
 ['[벌금형] 무면허운전, 음주운전, 0.168%로 위험운전치상',
  '벌금 800만원',
  'https://www.lawtalk.co.kr//posts/87764']]

데이터프레임 저장

In [8]:
import pandas as pd

df = pd.DataFrame(result, columns=['제목', '결과', '링크'])
df

,제목,결과,링크
0,[교통사고변호사] 위험운전치상 음주운전 벌금형 성공사례 ✔,벌금형,https://www.lawtalk.co.kr//posts/87800
1,✅음주 재범(수치 0.299) 벌금형 전문직 자격유지 #음주운전,벌금 1000만원,https://www.lawtalk.co.kr//posts/87777
2,"[벌금형] 무면허운전, 음주운전, 0.168%로 위험운전치상",벌금 800만원,https://www.lawtalk.co.kr//posts/87764


In [9]:
import openai
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

# OpenAI API 키 설정
client = openai.OpenAI(api_key = "api_key")

# 키워드 추출 함수 정의
def extract_keywords(title):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  
            messages=[
                {"role": "system", "content": "You are an assistant that extracts realistic search keywords from titles."},
                {"role": "user", "content": f"""Extract 3-5 unique search phrases that a suspect might use on Google, Naver, or YouTube from this title: '{title}'.
                Follow these rules:
                1. Do not repeat keywords.
                2. Create phrases of 1-3 words that sound natural for a search query.
                3. Phrases should reflect what a real person might actually type into a search engine.
                4. Provide only the phrases, separated by commas."""}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error extracting keywords for title '{title}': {str(e)}")
        return ""

# 기사 정보 추출
articles = soup.select('.card-list-section__card-wrap')
result = []
for article in articles:
    title = article.select_one('.card-list-section__card-wrap .solution-card__title').text.strip()
    law_result = article.select_one('.card-list-section__card-wrap .solution-card__sentence-result-text').text
    link = 'https://www.lawtalk.co.kr/' + article.select_one('.card-list-section__card-wrap .solution-card__link').attrs['href']
    result.append([title, law_result, link])

# DataFrame 생성
df = pd.DataFrame(result, columns=['제목', '결과', '링크'])

# 키워드 추출 및 새 컬럼에 추가
df['키워드'] = df['제목'].apply(extract_keywords)

df

,제목,결과,링크,키워드
0,[교통사고변호사] 위험운전치상 음주운전 벌금형 성공사례 ✔,벌금형,https://www.lawtalk.co.kr//posts/87800,"위험 운전 사례, 음주 운전 벌금, 교통 사고 변호사, 벌금형 성공 사례, 음주 운..."
1,✅음주 재범(수치 0.299) 벌금형 전문직 자격유지 #음주운전,벌금 1000만원,https://www.lawtalk.co.kr//posts/87777,"음주 재범 벌금, 전문직 자격 유지, 음주운전 처벌, 수치 0.299, 음주운전 법..."
2,"[벌금형] 무면허운전, 음주운전, 0.168%로 위험운전치상",벌금 800만원,https://www.lawtalk.co.kr//posts/87764,"무면허 운전 벌금, 음주 운전 0.168, 위험 운전 사실, 벌금형 사례, 음주 운..."
3,"음주운전 3회, 실형 불가피했지만 최종 집행유예",집행유예,https://www.lawtalk.co.kr//posts/87668,"음주운전 처벌, 실형 집행유예, 음주운전 3회, 최종 판결, 집행유예 조건"
4,"도주치상/사고후미조치, 혐의없음",혐의없음,https://www.lawtalk.co.kr//posts/87661,"도주치상, 사고후미조치, 혐의없음, 사고처리법, 법적절차"
...,...,...,...,...
2416,[태연법률사무소 성공사례] 운전면허정지 상담,승소,https://www.lawtalk.co.kr//posts/12607,"운전면허 정지 상담, 태연법률사무소 사례, 성공적인 법률 상담, 면허 재취득 방법,..."
2417,[뺑소니] 성공사례 - 무혐의,무혐의 처분,https://www.lawtalk.co.kr//posts/12312,"뺑소니 사례, 무혐의 성공, 뺑소니 사건, 성공적인 방어, 법률 조언"
2418,[교통사고처리특례법위반] 성공사례 -벌금형 감형,승소(항소인용),https://www.lawtalk.co.kr//posts/12309,"교통사고 벌금, 특례법 위반 사례, 성공 사례, 벌금형 감형 방법, 교통사고 처리 ..."
2419,[교통사고 범죄] 음주운전(도로교통법) 무죄사례,무죄선고,https://www.lawtalk.co.kr//posts/12114,"음주운전 사례, 교통사고 범죄, 무죄 판결, 도로교통법 정보, 음주운전 변호사"


In [10]:
df.to_excel('lotalk_음주운전.xlsx')